In [1]:
import pandas as pd
import numpy as np
import tabula

In [2]:
# Function to clean the pdf 
def func_clean_df(df):
    # The column Total Cost is the starting point for the new row...not the serial no...
    # get a new serial no ...increment if amount is present, else same 
    # so that we can group them into one row....
    newSrl = []
    srl = 0
    for i in range(0,len(df)):
        if pd.isna(df.loc[i,'Total Cost']) : # & pd.notna(df_2018_19.loc[i+1,'Total Cost']) :
            newSrl.append(srl)
        else:
            srl = srl + 1
            newSrl.append(srl)
    # Create a new column
    df['NewSrl'] = newSrl
    # Fill na with  0, so that can be summed in group
    df['Total Cost'] = df['Total Cost'].fillna(0)
    df = df.astype({'Total Cost': 'int32'})
    # Group and join the columns
    df1 = df.fillna('').groupby('NewSrl',as_index=False).agg(
        **{
            'Discipline': pd.NamedAgg('Discipline', ' '.join),
            'Sub Area': pd.NamedAgg('Sub Area', ' '.join),
            'Title': pd.NamedAgg('Title', ' '.join),
            'PI Details': pd.NamedAgg('PI Details', ' '.join),
            'Total Cost': pd.NamedAgg('Total Cost',sum)
          }   
          )    
    return(df1)

In [3]:
# Function to clean the pdf for 2015-16 as the column names and information is different
def func_clean_df2(df):
    # The column Total Cost is the starting point for the new row...not the serial no...
    # get a new serial no ...increment if amount is present, else same 
    # so that we can group them into one row....
    newSrl = []
    srl = 0
    for i in range(0,len(df)):
        if pd.isna(df.loc[i,'Total Cost']) : # & pd.notna(df_2018_19.loc[i+1,'Total Cost']) :
            newSrl.append(srl)
        else:
            srl = srl + 1
            newSrl.append(srl)
    # Create a new column
    df['NewSrl'] = newSrl
    # Fill na with  0, so that can be summed in group
    df['Total Cost'] = df['Total Cost'].fillna(0)
    df = df.astype({'Total Cost': 'int32'})
    # Group and join the columns
    df1 = df.fillna('').groupby('NewSrl',as_index=False).agg(
        **{
            'Scheme': pd.NamedAgg('Scheme', ' '.join),
            'Title': pd.NamedAgg('Title', ' '.join),
            'PI Name': pd.NamedAgg('PI Name', ' '.join),
            'Institute & Address': pd.NamedAgg('Institute & Address', ' '.join),
            'Total Cost': pd.NamedAgg('Total Cost',sum)
          }   
          )    
    return(df1)

# 1.  Process 2015-16 - pdf to csv

In [4]:
# read as one table, takes time 
dfs = tabula.read_pdf('data/Sanctioned Project EMR 2015-16.pdf',multiple_tables=False, pages="all")

df = dfs[0]
df.shape

(2608, 6)

In [5]:
df.columns

Index(['S.No.', 'Scheme', 'Title', 'PI Name', 'Institute & Address',
       'Total Cost'],
      dtype='object')

In [6]:
df_1516 = func_clean_df2(df)
df_1516.shape

(641, 6)

In [7]:
df_1516.head()

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
0,1,EMR,Development of Sugar amino\racid derived pepti...,Dr.Ravi Shankar\rAmpapathi,Central Drug Research\rInstitute Lucknow 22603...,2660000
1,2,EMR,Development of Sugar amino\racid derived pepti...,Prof.Tushar Kanti\rChakraborty,Indian Institute of Science\rBangalore 560012\...,6384000
2,3,EMR,Band gap tuning of transtion\rmetal oxides (Zn...,Dr.SanjibKarmakar,Gauhati University\rGuwahati 781014 Assam,1588800
3,4,EMR,Investigation of physical aging in\rHigh Tg po...,Dr.Asmita Sengupta,Visva -Bharati Central\rUniversity Bolpur\rSan...,1553520
4,5,EMR,Investigation of ultiferroic\rproperties of re...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman\rUniversity Chennai 600048\rT...,2355300


# 2. Process 2016-17 - pdf to csv

In [8]:
# read as one table, takes time 
dfs = tabula.read_pdf('data/EMR 2016-17.pdf',multiple_tables=False, pages="all")
df = dfs[0]
df.shape

(1750, 10)

In [9]:
# remove first 4 rows and last 4 columns
df = df.iloc[4:].copy()
# drop the nan columns
df.drop(['Unnamed: 6', 'Unnamed: 7','Unnamed: 8','Unnamed: 9'], axis = 1,inplace=True)
# rename columns
df.columns = ['S. No.', 'Discipline', 'Sub Area', 'Title', 'PI Details', 'Total Cost']
# re index after droppng rows
df.reset_index(drop=True, inplace=True)
df.head()

,S. No.,Discipline,Sub Area,Title,PI Details,Total Cost
0,1.,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on the effect of ionic\rliquids ...,"Dr. Harsh Kumar Manchanda,\rDepartment of Chem...",3300000
1,2.,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Optical And Electrochemical Sensors\rbased on ...,"Dr. Shweta Rana,\rDepartment of Chemistry,\rPa...",3025000
2,3.,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,A Concerted drive towards ambient\rpressure sy...,"Dr.Md. Motin Seikh,\rDepartment of Chemistry,\...",3739560
3,4.,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Estimation of nanoparticles (Toxic\rmetals) Fr...,"Dr. Kalpana Kumari,\rDepartment of Chemistry,\...",1900800
4,5.,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on an Electrochemical\rsystem co...,"Dr. Anudeep Kumar Narula,\rGuru Gobind Singh I...",3185600


In [10]:
df_1617 = func_clean_df(df)
df_1617.shape

(952, 6)

In [11]:
df_1617.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on the effect of ionic\rliquids ...,"Dr. Harsh Kumar Manchanda,\rDepartment of Chem...",3300000
1,2,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Optical And Electrochemical Sensors\rbased on ...,"Dr. Shweta Rana,\rDepartment of Chemistry,\rPa...",3025000
2,3,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,A Concerted drive towards ambient\rpressure sy...,"Dr.Md. Motin Seikh,\rDepartment of Chemistry,\...",3739560
3,4,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Estimation of nanoparticles (Toxic\rmetals) Fr...,"Dr. Kalpana Kumari,\rDepartment of Chemistry,\...",1900800
4,5,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on an Electrochemical\rsystem co...,"Dr. Anudeep Kumar Narula,\rGuru Gobind Singh I...",3185600


# 3. Process 2017-18 - pdf to csv

In [12]:
dfs = tabula.read_pdf('data/List of EMR projects funded by SERB during 2017-18.pdf',multiple_tables=False, pages="all")
df = dfs[0]
df.shape

Got stderr: Feb 15, 2022 6:34:18 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:18 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:20 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:21 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:25 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:26 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:27 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:28 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:29 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:41 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:42 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Feb 15, 2022 6:34:44 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



(3947, 6)

In [13]:
# first row is column names
# and then drop first row
df.columns = df.iloc[0]

df.drop(labels=0, axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,S. No,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Surface anchored metal organic\rthin film devices,"Dr. Nirmalya Ballav,\rIndian institute of\rSci...",6803280
1,2,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Design of diagnostic tools to\rmonitor certain...,"Dr. Meenakshi Singh,\rBanaras Hindu\rUniversit...",4387810
2,3,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Coordination complexes of\rorganic radicals sy...,"Dr. Prasanta Ghosh,\rRamakrishna Mission\rResi...",4163280
3,4,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Exploration of key photinduced\rdyanmics in in...,"Dr. Samir Kumar Pal, S\rN Bose National Centre...",1998000
4,5,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Metal Complexes based probes\rfor arylamine mo...,"Dr. Dr. Vaidyanathan\rGanesan, Central Leather...",2286145


In [14]:
df_1718 = func_clean_df(df)
df_1718.shape

(757, 6)

In [15]:
df_1718.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Surface anchored metal organic\rthin film devices,"Dr. Nirmalya Ballav,\rIndian institute of\rSci...",6803280
1,2,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Design of diagnostic tools to\rmonitor certain...,"Dr. Meenakshi Singh,\rBanaras Hindu\rUniversit...",4387810
2,3,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Coordination complexes of\rorganic radicals sy...,"Dr. Prasanta Ghosh,\rRamakrishna Mission\rResi...",4163280
3,4,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Exploration of key photinduced\rdyanmics in in...,"Dr. Samir Kumar Pal, S\rN Bose National Centre...",1998000
4,5,Chemical\rScience,Inorganic and\rPhysical\rChemistry,Metal Complexes based probes\rfor arylamine mo...,"Dr. Dr. Vaidyanathan\rGanesan, Central Leather...",2286145


# 4. Process 2018-19 - pdf to csv

In [16]:
# read as one table, takes time 
dfs = tabula.read_pdf('data/CRG 2018-19.pdf',multiple_tables=False, pages="all")

df = dfs[0]
df.shape

(10621, 6)

In [17]:
# first row is column names
# and then drop first row
df.columns = df.iloc[0]

df.drop(labels=0, axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,S. No.,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Photothermal Properties of\rAnisotropic Metal\...,"Dr. Sujit Kumar Ghosh,\rAssam University,\rSil...",2562560
1,2,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Highly Selective Ring-Opening\rPolymerization ...,"Dr. Tarun Kanti Panda,\rIndian Institute of\rT...",6508480
2,3,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,"Synthesis, Characterization And\rCatalytic App...","Dr. Firasat Hussain,\rUniversity of Delhi,\rDe...",3520000
3,4,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Integrated Gravity Flow Water\rFiltration Syst...,"Dr. Raj Kishore Patel,\rNational Institute of\...",3658600
4,5,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Novel Electrode Materials For\rReversible Alka...,"Dr. Preetam Singh,\rIndian Institute of\rTechn...",3665245


In [18]:
df_1819 = func_clean_df(df)
df_1819.shape

(1276, 6)

In [19]:
df_1819.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Photothermal Properties of\rAnisotropic Metal\...,"Dr. Sujit Kumar Ghosh,\rAssam University,\rSil...",2562560
1,2,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Highly Selective Ring-Opening\rPolymerization ...,"Dr. Tarun Kanti Panda,\rIndian Institute of\rT...",6508480
2,3,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,"Synthesis, Characterization And\rCatalytic App...","Dr. Firasat Hussain,\rUniversity of Delhi,\rDe...",3520000
3,4,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Integrated Gravity Flow Water\rFiltration Syst...,"Dr. Raj Kishore Patel,\rNational Institute of\...",3658600
4,5,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Novel Electrode Materials For\rReversible Alka...,"Dr. Preetam Singh,\rIndian Institute of\rTechn...",3665245


# 5. Process 2019-20 - pdf to csv

In [20]:
# read as one table, takes time 
dfs = tabula.read_pdf('data/CRG New Sanctioned Projects List 2019-20.pdf',multiple_tables=False, pages="all")

df = dfs[0]
df.shape

(6414, 6)

In [21]:
# first row is column names
# and then drop first row
df.columns = df.iloc[0]
df.drop(labels=0, axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,S.No,Discipline,Sub Area,Title,PI Details,Total Cost
0,1.,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Advanced Materials Towards\rElectrocatalytic O...,"Tharamani Chikka Nagaiah,\rIndian Institute of...",5783370
1,2.,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Chemistry In Self-Assembled\rCoordination Arch...,"Partha Sarathi Mukherjee,\rIndian Institute of...",5519890
2,3.,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Detoxification of Arsenic\rCompounds: Enzyme M...,"Gouriprasanna Roy,\rShiv Nadar University,\rGa...",5953775
3,4.,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,Investigation of Zintl phases as\refficient th...,"Manoj Raama Varma,\rNational Institute For\rIn...",3034800
4,5.,Chemical\rSciences,Inorganic and\rPhysical\rChemistry,High Potency of Complex Dietary\rΒ-Glucan Fibe...,"Durba Roy,\rBirla Institute of Technology\rAnd...",3666520


In [22]:
df_1920 = func_clean_df(df)
df_1920.shape

(1041, 6)

In [23]:
whos DataFrame

Variable   Type         Data/Info
---------------------------------
df         DataFrame    0      S.No          Disc<...>\n[6413 rows x 7 columns]
df_1516    DataFrame         NewSrl  Scheme      <...>n\n[641 rows x 6 columns]
df_1617    DataFrame         NewSrl           Dis<...>n\n[952 rows x 6 columns]
df_1718    DataFrame         NewSrl             D<...>n\n[757 rows x 6 columns]
df_1819    DataFrame          NewSrl             <...>\n[1276 rows x 6 columns]
df_1920    DataFrame          NewSrl             <...>\n[1041 rows x 6 columns]


In [24]:
# # Write the files
# df_1516.to_excel('processed/SERB_201516_2.xlsx', index=False)
# df_1617.to_excel('processed/SERB_201617.xlsx', index=False)
# df_1718.to_excel('processed/SERB_201718.xlsx', index=False)
# df_1819.to_excel('processed/SERB_201819.xlsx', index=False)
# df_1920.to_excel('processed/SERB_201920.xlsx', index=False)

# Clean  the df

***Check for nan and spaces in columns***

In [25]:
# Function to checkc for any na values and check for all spaces in columns -
def func_check_df(df):
    print('Any missing values? - ',df.isnull().values.any())
    print()
    print('Check  for num of blanks in columns :')
    for col in df.select_dtypes(include = ['object']):
        print(col,': ',df[col].str.isspace().sum())

# 1. Clean 2015-16

In [26]:
func_check_df(df_1516)

Any missing values? -  False

Check  for num of blanks in columns :
Scheme :  0
Title :  0
PI Name :  116
Institute & Address :  59


In [31]:
df_1516.head()

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
0,1,EMR,Development of Sugar amino\racid derived pepti...,Dr.Ravi Shankar\rAmpapathi,Central Drug Research\rInstitute Lucknow 22603...,2660000
1,2,EMR,Development of Sugar amino\racid derived pepti...,Prof.Tushar Kanti\rChakraborty,Indian Institute of Science\rBangalore 560012\...,6384000
2,3,EMR,Band gap tuning of transtion\rmetal oxides (Zn...,Dr.SanjibKarmakar,Gauhati University\rGuwahati 781014 Assam,1588800
3,4,EMR,Investigation of physical aging in\rHigh Tg po...,Dr.Asmita Sengupta,Visva -Bharati Central\rUniversity Bolpur\rSan...,1553520
4,5,EMR,Investigation of ultiferroic\rproperties of re...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman\rUniversity Chennai 600048\rT...,2355300


In [32]:
df_1516[df_1516['PI Name'].str.isspace()].head()

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
11,12,EMR,The bastar carton -the eastern Dr.Pritam Nasip...,,Indian Institute of Science Education and Rese...,2323000
12,13,EMR,Bioinventorying of Ascomycetus Dr.V.Venkateswa...,,Pondicherry University Puducherry 605014 Puduc...,3961200
13,14,EMR,Exploring Heat -stable Rubsico Dr.Ranjeet Ranj...,,Indian Agriculture Research Institute New Delh...,4833600
14,15,EMR,RNA-seq for discovery of novel Dr.Tripta Jhang...,,Central Institute of Medicinal & Aromatic Plan...,3768400
15,16,EMR,Integrated proteomic Dr.Asish Kumar metabolomi...,,Central Salt and Marine Chemical Research Inst...,2923000


***The name is embedded in title***

In [33]:
df_1516[df_1516['Institute & Address'].str.isspace()].head()

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
82,83,EMR,Long Term study of the Sun Dr.B.Ravindra using...,Indian Institute of Astrophysics Bangalore 560...,,2228400
83,84,EMR,Development of novel Dr.N.Rajendran nanostruct...,Anna University Chennai 600025 Tamilnadu,,3456800
84,85,EMR,Efficient poly (ethylene oxide ) Dr.Arul Manue...,Central Electrochemical Research Institute Kar...,,3771000
85,86,EMR,Magneto-optic trapping of Dr.Umakant Damodar I...,Education and Research Pune 411008 Maharashtra,,5480000
86,87,EMR,Atomistic simulations of boron Dr.Sridhar Sahu...,Indian School of Mines Dhanbad 826004 Jharkhand,,2523840


***The  institue name is in PI name, and the PI name is embedded in the Title***
________________________________________________________________
***Clean the Title, PI Name and Institue and Address manually***

In [ ]:
df_1516.shape

In [34]:
df_1516 = pd.read_excel('processed\ed\SERB_15_16V2.xlsx')
df_1516.shape

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
0,1,EMR,Development of Sugar amino_x000D_acid derived ...,Dr.Ravi Shankar_x000D_Ampapathi,Central Drug Research_x000D_Institute Lucknow ...,2660000
1,2,EMR,Development of Sugar amino_x000D_acid derived ...,Prof.Tushar Kanti_x000D_Chakraborty,Indian Institute of Science_x000D_Bangalore 56...,6384000
2,3,EMR,Band gap tuning of transtion_x000D_metal oxide...,Dr.SanjibKarmakar,Gauhati University_x000D_Guwahati 781014 Assam,1588800
3,4,EMR,Investigation of physical aging in_x000D_High ...,Dr.Asmita Sengupta,Visva -Bharati Central_x000D_University Bolpur...,1553520
4,5,EMR,Investigation of ultiferroic_x000D_properties ...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman_x000D_University Chennai 6000...,2355300


In [ ]:
df_1516.head()

In [35]:
func_check_df(df_1516)

Any missing values? -  False

Check  for num of blanks in columns :
Scheme :  0
Title :  0
PI Name :  0
Institute & Address :  0


In [36]:
df_1516['Title'] = df_1516['Title'].replace('_x000D_', ' ', regex=True)
df_1516['PI Name'] = df_1516['PI Name'].replace('_x000D_', ' ', regex=True)
df_1516['Institute & Address'] = df_1516['Institute & Address'].replace('_x000D_', ' ', regex=True)
df_1516.head()

,NewSrl,Scheme,Title,PI Name,Institute & Address,Total Cost
0,1,EMR,Development of Sugar amino acid derived peptid...,Dr.Ravi Shankar Ampapathi,Central Drug Research Institute Lucknow 226031...,2660000
1,2,EMR,Development of Sugar amino acid derived peptid...,Prof.Tushar Kanti Chakraborty,Indian Institute of Science Bangalore 560012 K...,6384000
2,3,EMR,Band gap tuning of transtion metal oxides (ZnO...,Dr.SanjibKarmakar,Gauhati University Guwahati 781014 Assam,1588800
3,4,EMR,Investigation of physical aging in High Tg pol...,Dr.Asmita Sengupta,Visva -Bharati Central University Bolpur Santi...,1553520
4,5,EMR,Investigation of ultiferroic properties of rer...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman University Chennai 600048 Tam...,2355300


In [37]:
df_1516['Scheme'].value_counts()

EMR              226
EMR              152
EMR              134
EMR               49
EMR               45
EMR               23
EMR                6
EMR                3
EMR                1
EMR                1
EMR                1
Name: Scheme, dtype: int64

In [38]:
# Remove spaces 
df_1516['Scheme'] = df_1516['Scheme'].str.strip()
df_1516['Title'] = df_1516['Title'].str.strip()
df_1516['PI Name'] = df_1516['PI Name'].str.strip()
df_1516['Institute & Address'] = df_1516['Institute & Address'].str.strip()

In [39]:
# Rename Scheme to  Discipline
df_1516.rename({'Scheme': 'Discipline'}, axis=1, inplace=True)
# Add SubArea , all other years file has this column
df_1516['Sub Area'] = 'EMR'
df_1516.head()

,NewSrl,Discipline,Title,PI Name,Institute & Address,Total Cost,Sub Area
0,1,EMR,Development of Sugar amino acid derived peptid...,Dr.Ravi Shankar Ampapathi,Central Drug Research Institute Lucknow 226031...,2660000,EMR
1,2,EMR,Development of Sugar amino acid derived peptid...,Prof.Tushar Kanti Chakraborty,Indian Institute of Science Bangalore 560012 K...,6384000,EMR
2,3,EMR,Band gap tuning of transtion metal oxides (ZnO...,Dr.SanjibKarmakar,Gauhati University Guwahati 781014 Assam,1588800,EMR
3,4,EMR,Investigation of physical aging in High Tg pol...,Dr.Asmita Sengupta,Visva -Bharati Central University Bolpur Santi...,1553520,EMR
4,5,EMR,Investigation of ultiferroic properties of rer...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman University Chennai 600048 Tam...,2355300,EMR


In [40]:
# Rename PI Name to Professor
df_1516.rename({'PI Name': 'Professor'}, axis=1, inplace=True)
# Rename Institue and Address to PI Details
df_1516.rename({'Institute & Address': 'PI Details'}, axis=1, inplace=True)

In [52]:
# Copy PI Deatils to professor
df_1516['Institute'] = df_1516['PI Details']

In [53]:
df_1516.head()

,NewSrl,Discipline,Title,Professor,PI Details,Total Cost,Sub Area,Institute
0,1,EMR,Development of Sugar amino acid derived peptid...,Dr.Ravi Shankar Ampapathi,Central Drug Research Institute Lucknow 226031...,2660000,EMR,Central Drug Research Institute Lucknow 226031...
1,2,EMR,Development of Sugar amino acid derived peptid...,Prof.Tushar Kanti Chakraborty,Indian Institute of Science Bangalore 560012 K...,6384000,EMR,Indian Institute of Science Bangalore 560012 K...
2,3,EMR,Band gap tuning of transtion metal oxides (ZnO...,Dr.SanjibKarmakar,Gauhati University Guwahati 781014 Assam,1588800,EMR,Gauhati University Guwahati 781014 Assam
3,4,EMR,Investigation of physical aging in High Tg pol...,Dr.Asmita Sengupta,Visva -Bharati Central University Bolpur Santi...,1553520,EMR,Visva -Bharati Central University Bolpur Santi...
4,5,EMR,Investigation of ultiferroic properties of rer...,Dr.I.B.Shameem Banu,B.S.Abdur Rahman University Chennai 600048 Tam...,2355300,EMR,B.S.Abdur Rahman University Chennai 600048 Tam...


# 2. Clean 2016-17

In [27]:
func_check_df(df_1617)

Any missing values? -  False

Check  for num of blanks in columns :
Discipline :  108
Sub Area :  0
Title :  0
PI Details :  0


In [42]:
df_1617.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on the effect of ionic\rliquids ...,"Dr. Harsh Kumar Manchanda,\rDepartment of Chem...",3300000
1,2,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Optical And Electrochemical Sensors\rbased on ...,"Dr. Shweta Rana,\rDepartment of Chemistry,\rPa...",3025000
2,3,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,A Concerted drive towards ambient\rpressure sy...,"Dr.Md. Motin Seikh,\rDepartment of Chemistry,\...",3739560
3,4,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Estimation of nanoparticles (Toxic\rmetals) Fr...,"Dr. Kalpana Kumari,\rDepartment of Chemistry,\...",1900800
4,5,Chemical\rSciences,Inorganic &\rPhysical\rChemistry,Investigation on an Electrochemical\rsystem co...,"Dr. Anudeep Kumar Narula,\rGuru Gobind Singh I...",3185600


In [43]:
df_1617[df_1617['Discipline'].str.isspace()].head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
96,97,,Inorganic & Physical Chemistry,Synthesis and characterization of chitosan bas...,"Prof. Sankaran Meenakshi, Department of Chemis...",1909600
97,98,,Inorganic & Physical Chemistry,Molecular recognition receptors for biomarkers...,"Dr. Amitava Das, Department of Chemistry, Cent...",7354370
98,99,,Inorganic & Physical Chemistry,Temperature dependent semiconductor properties...,"Dr. Pankaj Kumar Mandal, Department of Chemist...",8575910
99,100,,Inorganic & Physical Chemistry,Tailoring rare earth element additions in zirc...,"Dr. Kannan Sanjeevi, Centre For Nano Science a...",4274600
100,101,,Inorganic & Physical Chemistry,Continuous Flow An Attempt to miniaturization ...,"Dr. Ramesh C. Deka, Department of Chemical Sci...",4740120


In [44]:
# All spaces in Discipline are Chemical Sciences
df_1617['Discipline'] = np.where(df_1617['Discipline'].str.isspace(),'Chemical Sciences',df_1617['Discipline'])

In [46]:
# Remove spaces
df_1617['Discipline'] = df_1617['Discipline'].str.strip()
df_1617['Sub Area'] = df_1617['Sub Area'].str.strip()
df_1617['Title'] = df_1617['Title'].str.strip()
df_1617['PI Details'] = df_1617['PI Details'].str.strip()

In [48]:
# Remove all carriage return
df_1617.replace(r'\r', ' ', regex = True, inplace = True)

In [49]:
# Split PI to get professor and Institutes name,  if not str[0], we get a list 
df_1617['Professor'] = df_1617['PI Details'].str.split(',').str[0]
df_1617['Institute'] = df_1617['PI Details'].str.split(',',1).str[1]
df_1617.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost,Professor,Institute
0,1,Chemical Sciences,Inorganic & Physical Chemistry,Investigation on the effect of ionic liquids o...,"Dr. Harsh Kumar Manchanda, Department of Chemi...",3300000,Dr. Harsh Kumar Manchanda,"Department of Chemistry, Dr.B R Ambedkar Nati..."
1,2,Chemical Sciences,Inorganic & Physical Chemistry,Optical And Electrochemical Sensors based on B...,"Dr. Shweta Rana, Department of Chemistry, Panj...",3025000,Dr. Shweta Rana,"Department of Chemistry, Panjab University, C..."
2,3,Chemical Sciences,Inorganic & Physical Chemistry,A Concerted drive towards ambient pressure syn...,"Dr.Md. Motin Seikh, Department of Chemistry, V...",3739560,Dr.Md. Motin Seikh,"Department of Chemistry, Visva Bharati Univer..."
3,4,Chemical Sciences,Inorganic & Physical Chemistry,Estimation of nanoparticles (Toxic metals) Fro...,"Dr. Kalpana Kumari, Department of Chemistry, B...",1900800,Dr. Kalpana Kumari,"Department of Chemistry, Birchand Patel Smara..."
4,5,Chemical Sciences,Inorganic & Physical Chemistry,Investigation on an Electrochemical system com...,"Dr. Anudeep Kumar Narula, Guru Gobind Singh In...",3185600,Dr. Anudeep Kumar Narula,"Guru Gobind Singh Indraprastha University,New..."


In [54]:
df_1516.columns

Index(['NewSrl', 'Discipline', 'Title', 'Professor', 'PI Details',
       'Total Cost', 'Sub Area', 'Institute'],
      dtype='object')

In [51]:
df_1617.columns

Index(['NewSrl', 'Discipline', 'Sub Area', 'Title', 'PI Details', 'Total Cost',
       'Professor', 'Institute'],
      dtype='object')

# 3. Clean 2017-18

In [55]:
func_check_df(df_1718)

Any missing values? -  False

Check  for num of blanks in columns :
Discipline :  0
Sub Area :  0
Title :  0
PI Details :  0


In [56]:
df_1718['Discipline'] = df_1718['Discipline'].str.strip()
df_1718['Sub Area'] = df_1718['Sub Area'].str.strip()
df_1718['Title'] = df_1718['Title'].str.strip()
df_1718['PI Details'] = df_1718['PI Details'].str.strip()
# remove carriage return
df_1718.replace(r'\r', ' ', regex = True, inplace = True)
df_1718.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical Science,Inorganic and Physical Chemistry,Surface anchored metal organic thin film devices,"Dr. Nirmalya Ballav, Indian institute of Scien...",6803280
1,2,Chemical Science,Inorganic and Physical Chemistry,Design of diagnostic tools to monitor certain ...,"Dr. Meenakshi Singh, Banaras Hindu University,...",4387810
2,3,Chemical Science,Inorganic and Physical Chemistry,Coordination complexes of organic radicals sys...,"Dr. Prasanta Ghosh, Ramakrishna Mission Reside...",4163280
3,4,Chemical Science,Inorganic and Physical Chemistry,Exploration of key photinduced dyanmics in ino...,"Dr. Samir Kumar Pal, S N Bose National Centre ...",1998000
4,5,Chemical Science,Inorganic and Physical Chemistry,Metal Complexes based probes for arylamine mod...,"Dr. Dr. Vaidyanathan Ganesan, Central Leather ...",2286145


In [57]:
# Split PI to get professor and Institutes name,  if not str[0], we get a list 
df_1718['Professor'] = df_1718['PI Details'].str.split(',').str[0]
df_1718['Institute'] = df_1718['PI Details'].str.split(',',1).str[1]
df_1718.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost,Professor,Institute
0,1,Chemical Science,Inorganic and Physical Chemistry,Surface anchored metal organic thin film devices,"Dr. Nirmalya Ballav, Indian institute of Scien...",6803280,Dr. Nirmalya Ballav,Indian institute of Science Education and Res...
1,2,Chemical Science,Inorganic and Physical Chemistry,Design of diagnostic tools to monitor certain ...,"Dr. Meenakshi Singh, Banaras Hindu University,...",4387810,Dr. Meenakshi Singh,"Banaras Hindu University, Varanasi, Uttar Pra..."
2,3,Chemical Science,Inorganic and Physical Chemistry,Coordination complexes of organic radicals sys...,"Dr. Prasanta Ghosh, Ramakrishna Mission Reside...",4163280,Dr. Prasanta Ghosh,"Ramakrishna Mission Residential College, Kolk..."
3,4,Chemical Science,Inorganic and Physical Chemistry,Exploration of key photinduced dyanmics in ino...,"Dr. Samir Kumar Pal, S N Bose National Centre ...",1998000,Dr. Samir Kumar Pal,"S N Bose National Centre for Basic Sciences, ..."
4,5,Chemical Science,Inorganic and Physical Chemistry,Metal Complexes based probes for arylamine mod...,"Dr. Dr. Vaidyanathan Ganesan, Central Leather ...",2286145,Dr. Dr. Vaidyanathan Ganesan,"Central Leather Research institute, Chennai, ..."


# 4. Clean 2018-19

In [58]:
func_check_df(df_1819)

Any missing values? -  False

Check  for num of blanks in columns :
Discipline :  0
Sub Area :  0
Title :  0
PI Details :  0


In [59]:
df_1819['Discipline'] = df_1819['Discipline'].str.strip()
df_1819['Sub Area'] = df_1819['Sub Area'].str.strip()
df_1819['Title'] = df_1819['Title'].str.strip()
df_1819['PI Details'] = df_1819['PI Details'].str.strip()
df_1819.replace(r'\r', ' ', regex = True, inplace = True)
df_1819.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost
0,1,Chemical Sciences,Inorganic and Physical Chemistry,Photothermal Properties of Anisotropic Metal N...,"Dr. Sujit Kumar Ghosh, Assam University, Silch...",2562560
1,2,Chemical Sciences,Inorganic and Physical Chemistry,Highly Selective Ring-Opening Polymerization o...,"Dr. Tarun Kanti Panda, Indian Institute of Tec...",6508480
2,3,Chemical Sciences,Inorganic and Physical Chemistry,"Synthesis, Characterization And Catalytic Appl...","Dr. Firasat Hussain, University of Delhi, Delh...",3520000
3,4,Chemical Sciences,Inorganic and Physical Chemistry,Integrated Gravity Flow Water Filtration Syste...,"Dr. Raj Kishore Patel, National Institute of T...",3658600
4,5,Chemical Sciences,Inorganic and Physical Chemistry,Novel Electrode Materials For Reversible Alkal...,"Dr. Preetam Singh, Indian Institute of Technol...",3665245


In [60]:
# Split PI to get professor and Institutes name,  if not str[0], we get a list 
df_1819['Professor'] = df_1819['PI Details'].str.split(',').str[0]
df_1819['Institute'] = df_1819['PI Details'].str.split(',',1).str[1]
df_1819.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost,Professor,Institute
0,1,Chemical Sciences,Inorganic and Physical Chemistry,Photothermal Properties of Anisotropic Metal N...,"Dr. Sujit Kumar Ghosh, Assam University, Silch...",2562560,Dr. Sujit Kumar Ghosh,"Assam University, Silchar, Assam, 788011"
1,2,Chemical Sciences,Inorganic and Physical Chemistry,Highly Selective Ring-Opening Polymerization o...,"Dr. Tarun Kanti Panda, Indian Institute of Tec...",6508480,Dr. Tarun Kanti Panda,"Indian Institute of Technology Hyderabad, Hyd..."
2,3,Chemical Sciences,Inorganic and Physical Chemistry,"Synthesis, Characterization And Catalytic Appl...","Dr. Firasat Hussain, University of Delhi, Delh...",3520000,Dr. Firasat Hussain,"University of Delhi, Delhi, Delhi, 110007"
3,4,Chemical Sciences,Inorganic and Physical Chemistry,Integrated Gravity Flow Water Filtration Syste...,"Dr. Raj Kishore Patel, National Institute of T...",3658600,Dr. Raj Kishore Patel,"National Institute of Technology Rourkela, Ro..."
4,5,Chemical Sciences,Inorganic and Physical Chemistry,Novel Electrode Materials For Reversible Alkal...,"Dr. Preetam Singh, Indian Institute of Technol...",3665245,Dr. Preetam Singh,"Indian Institute of Technology (Bhu), Varanas..."


# 5. Clean 2019-20

In [30]:
func_check_df(df_1920)

Any missing values? -  False

Check  for num of blanks in columns :
Discipline :  0
Sub Area :  0
Title :  0
PI Details :  0


In [61]:
df_1920['Discipline'] = df_1920['Discipline'].str.strip()
df_1920['Sub Area'] = df_1920['Sub Area'].str.strip()
df_1920['Title'] = df_1920['Title'].str.strip()
df_1920['PI Details'] = df_1920['PI Details'].str.strip()
df_1920.replace(r'\r', ' ', regex = True, inplace = True)

In [62]:
# Split PI to get professor and Institutes name,  if not str[0], we get a list 
df_1920['Professor'] = df_1920['PI Details'].str.split(',').str[0]
df_1920['Institute'] = df_1920['PI Details'].str.split(',',1).str[1]
df_1920.head()

,NewSrl,Discipline,Sub Area,Title,PI Details,Total Cost,Professor,Institute
0,1,Chemical Sciences,Inorganic and Physical Chemistry,Advanced Materials Towards Electrocatalytic Ox...,"Tharamani Chikka Nagaiah, Indian Institute of ...",5783370,Tharamani Chikka Nagaiah,"Indian Institute of Technology Ropar, Ropar,P..."
1,2,Chemical Sciences,Inorganic and Physical Chemistry,Chemistry In Self-Assembled Coordination Archi...,"Partha Sarathi Mukherjee, Indian Institute of ...",5519890,Partha Sarathi Mukherjee,"Indian Institute of Science,Bangalore, Bangal..."
2,3,Chemical Sciences,Inorganic and Physical Chemistry,Detoxification of Arsenic Compounds: Enzyme Mi...,"Gouriprasanna Roy, Shiv Nadar University, Gaut...",5953775,Gouriprasanna Roy,"Shiv Nadar University, Gautambudh Nagar,Uttar..."
3,4,Chemical Sciences,Inorganic and Physical Chemistry,Investigation of Zintl phases as efficient the...,"Manoj Raama Varma, National Institute For Inte...",3034800,Manoj Raama Varma,National Institute For Interdisciplinary Scie...
4,5,Chemical Sciences,Inorganic and Physical Chemistry,High Potency of Complex Dietary Β-Glucan Fiber...,"Durba Roy, Birla Institute of Technology And S...",3666520,Durba Roy,Birla Institute of Technology And Science Pil...


# Merge all and then clean Disciplinne, sub area and get pic cd and State

In [63]:
df_1516['Year'] = '2015-16'
df_1617['Year'] = '2016-17'
df_1718['Year'] = '2017-18'
df_1819['Year'] = '2018-19'
df_1920['Year'] = '2019-20'

In [64]:
# The cols sequence is based on  first data frame 
df = pd.concat([df_1920,df_1819,df_1718,df_1617,df_1516],ignore_index=True)
df.drop(['NewSrl'], axis = 1,inplace=True)
#df.rename({'pin_cd': 'Pin Code'}, axis=1, inplace=True)
#pd.set_option("precision", 1)
df.shape

(4667, 8)

In [65]:
df.head()

,Discipline,Sub Area,Title,PI Details,Total Cost,Professor,Institute,Year
0,Chemical Sciences,Inorganic and Physical Chemistry,Advanced Materials Towards Electrocatalytic Ox...,"Tharamani Chikka Nagaiah, Indian Institute of ...",5783370,Tharamani Chikka Nagaiah,"Indian Institute of Technology Ropar, Ropar,P...",2019-20
1,Chemical Sciences,Inorganic and Physical Chemistry,Chemistry In Self-Assembled Coordination Archi...,"Partha Sarathi Mukherjee, Indian Institute of ...",5519890,Partha Sarathi Mukherjee,"Indian Institute of Science,Bangalore, Bangal...",2019-20
2,Chemical Sciences,Inorganic and Physical Chemistry,Detoxification of Arsenic Compounds: Enzyme Mi...,"Gouriprasanna Roy, Shiv Nadar University, Gaut...",5953775,Gouriprasanna Roy,"Shiv Nadar University, Gautambudh Nagar,Uttar...",2019-20
3,Chemical Sciences,Inorganic and Physical Chemistry,Investigation of Zintl phases as efficient the...,"Manoj Raama Varma, National Institute For Inte...",3034800,Manoj Raama Varma,National Institute For Interdisciplinary Scie...,2019-20
4,Chemical Sciences,Inorganic and Physical Chemistry,High Potency of Complex Dietary Β-Glucan Fiber...,"Durba Roy, Birla Institute of Technology And S...",3666520,Durba Roy,Birla Institute of Technology And Science Pil...,2019-20


# Clean Discipline

In [66]:
df.isna().sum()

Discipline    0
Sub Area      0
Title         0
PI Details    0
Total Cost    0
Professor     0
Institute     0
Year          0
dtype: int64

In [67]:
df['Discipline'].value_counts()

Life Sciences                      1095
EMR                                 641
Engineering Sciences                404
Chemical Sciences                   403
Life  Sciences                      366
Physical Sciences                   287
Chemical  Sciences                  201
Engineering  Sciences               189
Phy sical Sciences                  172
Chemical Science                    172
Physical  Sciences                  171
Eng ineering Sciences               137
L ife Sciences                      133
Earth & Atmospheric Sciences         62
Earth &  Atmospheric  Sciences       55
Mathematical Sciences                53
Mathematical  Sciences               47
Ma thematical Sciences               38
Ear th and Atmospheric Sciences      33
Combustion                            6
busion                                2
Name: Discipline, dtype: int64

In [68]:
# From 1920
df['Discipline'] = df['Discipline'].str.replace('L ife Sciences', 'Life Sciences')
# From1819
df['Discipline'] = df['Discipline'].str.replace('Physical  Sciences', 'Physical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Engineering  Sciences', 'Engineering Sciences')
df['Discipline'] = df['Discipline'].str.replace('Chemical  Sciences', 'Chemical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Mathematical  Sciences', 'Mathematical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Earth &  Atmospheric  Sciences', 'Earth & Atmospheric Sciences')
# From 1718
df['Discipline'] = df['Discipline'].str.replace('Chemical Science', 'Chemical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Combustion', 'Chemical Sciences')
#From 1617
df['Discipline'] = df['Discipline'].str.replace('Life  Sciences', 'Life Sciences')
df['Discipline'] = df['Discipline'].str.replace('Phy sical Sciences', 'Physical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Eng ineering Sciences', 'Engineering Sciences')
#df_1617['Discipline'] = df_1617['Discipline'].str.replace('Chemical Science', 'Chemical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Ma thematical Sciences', 'Mathematical Sciences')
df['Discipline'] = df['Discipline'].str.replace('Ear th and Atmospheric Sciences', 'Earth & Atmospheric Sciences')
df['Discipline'] = df['Discipline'].str.replace('busion', 'Chemical Sciences')

In [69]:
df['Discipline'].value_counts()

Life Sciences                   1594
Engineering Sciences             730
EMR                              641
Physical Sciences                630
Chemical Sciencess               604
Chemical Sciences                180
Earth & Atmospheric Sciences     150
Mathematical Sciences            138
Name: Discipline, dtype: int64

# Clean  Sub Area

In [70]:
df['Sub Area'].value_counts()

EMR                                                               641
Physical Sciences                                                 459
Health Sciences                                                   339
Plant Sciences                                                    250
Inorganic and Physical Chemistry                                  229
Health  Sciences                                                  223
Organic Chemistry                                                 218
Electrical, Electronics & Computer Engineering                    193
Physical  Sciences                                                171
Animal Sciences                                                   129
Animal  Sciences                                                  129
Inorganic & Physical Chemistry                                    128
Inorganic and  Physical  Chemistry                                116
Materials, Mining & Minerals Engineering                          111
Biophysics,  Biochem

In [71]:
# From 1920
df['Sub Area'] = df['Sub Area'].str.replace('Organismal And Evolutionary  Biology (Palant Science)', 
                                                      'Organismal And Evolutionary Biology (Plant Science)',regex=False)

df['Sub Area'] = df['Sub Area'].str.replace('Organismal And Evolutionary Biology (Palant Science)', 
                                                      'Organismal And Evolutionary Biology (Plant Science)',regex=False)

df['Sub Area'] = df['Sub Area'].str.replace('Electrical, Electronics &  Computer Engineering', 
                                                      'Electrical, Electronics & Computer Engineering',regex=False)

# From 1819
#to be done...not done previously...allneeds to be edited....

# from 1718
# Nothing

# From 1617
df['Sub Area'] = df['Sub Area'].str.replace('Combusion', 
                                                      'Inorganic & Physical Chemistry',regex=False)

# 

In [72]:
df['Sub Area'].value_counts()

EMR                                                               641
Physical Sciences                                                 459
Health Sciences                                                   339
Plant Sciences                                                    250
Inorganic and Physical Chemistry                                  229
Health  Sciences                                                  223
Organic Chemistry                                                 218
Electrical, Electronics & Computer Engineering                    214
Physical  Sciences                                                171
Inorganic & Physical Chemistry                                    130
Animal  Sciences                                                  129
Animal Sciences                                                   129
Inorganic and  Physical  Chemistry                                116
Materials, Mining & Minerals Engineering                          111
Biophysics,  Biochem

In [ ]:
# Clean soome more of sub arae

# Not get pin code and State